In [1]:
suppressPackageStartupMessages(library(cicero))
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(data.table))
suppressPackageStartupMessages(library(parallel))

In [2]:
wd = '/nfs/lab/projects/lung_snATAC_tempJN/ciceroPooled'
matrixFile = '/nfs/lab/projects/lung_snATAC_tempJN/joshsCiceroOutput/splitReformattedMatrixFiles/lung_snATAC_new_chr1.singleChrom.lngfmtMtx.mtx'
clusterFile = '/nfs/lab/projects/lung_snATAC_tempJN/joshsCiceroOutput/splitClusterLabelFiles/lung_snATAC.merged_samples.ALL.singleChrom.chr1.cluster_labels.txt'
outConnectionFilePrefix = 'lung_snATAC_new.chr1.'
outRdsConnsSuffix = '.1MB_cicero_conns.rds'
outConnsSuffix = '.cicero_conns.txt'
outConnsDedupSuffix = '.cicero_conns_dedup.txt'

In [3]:
clus <- makeCluster(32)
#clusterExport(clus)

In [4]:
setwd(wd)

In [5]:
data = read.table(matrixFile,header=T,  sep="\t") ### long for matrix peak x barcode

In [6]:
head(data)
dim(data)

peak,cell,value
<fct>,<fct>,<int>
chr1_100003518_100003891,D032-2_AAACGCAAGCCTGTCTACACGA,2
chr1_100003518_100003891,D032-2_AGTGCCTTGAGTTGCGAATGGG,2
chr1_100003518_100003891,D032-2_ATCGTACTGTTCTGACGGGACT,4
chr1_100003518_100003891,D032-2_CAATCTACGACTCCGATATGAC,2
chr1_100003518_100003891,D032-2_CACAATGCCGTGACACTGTGCG,2
chr1_100003518_100003891,D032-2_CAGGCCTGTAGATATGGTGCCA,2


[1] 20412742        3

In [8]:
#head(data)
### convert to sparse matrix peak x barcode
colnames(data)[colnames(data) == 'cell'] <- 'barcode'
sc.data <- with(data, sparseMatrix(i=as.numeric(peak), j=as.numeric(barcode), 
                                   x=value, dimnames=list(levels(peak), levels(barcode))))

In [9]:
head(sc.data)
dim(sc.data)

   [[ suppressing 34 column names 'D032_AACGAGAGCTAACCACCTAAAG', 'D032_AACGAGAGCTAACCATGCATGA', 'D032_AACGAGAGCTAACGTGATTTGG' ... ]]


6 x 90965 sparse Matrix of class "dgCMatrix"
                                                                              
chr1_100003518_100003891 . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1_100009551_100010651 . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1_100012339_100012539 . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1_100014582_100014865 . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1_100019342_100019752 . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1_100024645_100025217 . . . . . . . . . . . . . . . . . . . . . . . . . . .
                                             
chr1_100003518_100003891 . . . . . . . ......
chr1_100009551_100010651 . . . . . . . ......
chr1_100012339_100012539 . . . . . . . ......
chr1_100014582_100014865 . . . . . . . ......
chr1_100019342_100019752 . . . . . . . ......
chr1_100024645_100025217 . . . . . . . ......

 .....suppressing 90931 columns in show(); maybe adjust 'options(max.print= *, 

[1] 34428 90965

In [14]:
sc.umap <- read.table( clusterFile, header=T, row.names=1) ### map barcode to cluster 
colnames(sc.umap)[colnames(sc.umap) == 'all'] <- 'cluster_name'

In [15]:
head(sc.umap)
dim(sc.umap)

,UMAP1,UMAP2,subset2,cluster_name,log_usable_counts
,<dbl>,<dbl>,<fct>,<fct>,<dbl>
D062_AACGAGAGCTAAACCTAAGTGG,-3.965895,1.0771579,16,all,6.184149
D062_AACGAGAGCTAAACTAGCCCTA,2.340209,14.4901056,"4,0",all,8.326275
D062_AACGAGAGCTAAAGGAACAGAC,7.979088,1.6915314,3,all,6.549651
D062_AACGAGAGCTAAAGGATCGGCT,3.531774,14.2219507,"4,0",all,7.875879
D062_AACGAGAGCTAAGTGTCGTGGA,-11.856324,-0.8003238,0,all,7.232733
D062_AACGAGAGCTAATCCCGCGCTT,6.858509,2.9069133,3,all,6.364751


[1] 90965     5

In [16]:
sc.umap.subset <- sc.umap
sc.data.subset <- sc.data
#sc.data.subset <- sc.data.subset[ ,colnames(sc.data.subset) %in% rownames(sc.umap.subset)] > 0



In [17]:

#rowsCount <- nrow(sc.data.subset)
#rowsToUse <- rowsCount * 0.25


#sc.data.subset <- sc.data.subset[1:rowsToUse,]

In [18]:
head(sc.data.subset)
dim(sc.data.subset)

   [[ suppressing 34 column names 'D032_AACGAGAGCTAACCACCTAAAG', 'D032_AACGAGAGCTAACCATGCATGA', 'D032_AACGAGAGCTAACGTGATTTGG' ... ]]


6 x 90965 sparse Matrix of class "dgCMatrix"
                                                                              
chr1_100003518_100003891 . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1_100009551_100010651 . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1_100012339_100012539 . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1_100014582_100014865 . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1_100019342_100019752 . . . . . . . . . . . . . . . . . . . . . . . . . . .
chr1_100024645_100025217 . . . . . . . . . . . . . . . . . . . . . . . . . . .
                                             
chr1_100003518_100003891 . . . . . . . ......
chr1_100009551_100010651 . . . . . . . ......
chr1_100012339_100012539 . . . . . . . ......
chr1_100014582_100014865 . . . . . . . ......
chr1_100019342_100019752 . . . . . . . ......
chr1_100024645_100025217 . . . . . . . ......

 .....suppressing 90931 columns in show(); maybe adjust 'options(max.print= *, 

[1] 34428 90965

In [19]:
cellinfo <-data.frame(cells=colnames(sc.data.subset))
row.names(cellinfo) <- cellinfo$cells
dhsinfo <- data.frame(site_name=rownames(sc.data.subset))
row.names(dhsinfo) <- dhsinfo$site_name
dhsinfo <- cbind(dhsinfo, stringr::str_split_fixed(dhsinfo$site_name, "_", 3))
names(dhsinfo) <- c('site_name','chr','bp1','bp2')
dhsinfo$chr <- gsub('chr','', dhsinfo$chr)
dhsinfo$bp1 <- as.numeric(as.character(dhsinfo$bp1))
dhsinfo$bp2 <- as.numeric(as.character(dhsinfo$bp2))


In [20]:
input_cds <- suppressWarnings(newCellDataSet(as(sc.data.subset, 'dgCMatrix'),
                            phenoData = methods::new('AnnotatedDataFrame', data = cellinfo),
                            featureData = methods::new('AnnotatedDataFrame', data = dhsinfo),
                            expressionFamily=negbinomial.size(),
                            lowerDetectionLimit=0))

In [21]:
head(fData(input_cds))

,site_name,chr,bp1,bp2
,<fct>,<chr>,<dbl>,<dbl>
chr1_100003518_100003891,chr1_100003518_100003891,1,100003518,100003891
chr1_100009551_100010651,chr1_100009551_100010651,1,100009551,100010651
chr1_100012339_100012539,chr1_100012339_100012539,1,100012339,100012539
chr1_100014582_100014865,chr1_100014582_100014865,1,100014582,100014865
chr1_100019342_100019752,chr1_100019342_100019752,1,100019342,100019752
chr1_100024645_100025217,chr1_100024645_100025217,1,100024645,100025217


In [23]:
input_cds@expressionFamily <- binomialff()
input_cds@expressionFamily@vfamily <- 'binomialff'
input_cds <- detectGenes(input_cds)
input_cds <- estimateSizeFactors(input_cds)
input_cds <- input_cds[fData(input_cds)$num_cells_expressed > 0,]


 head(fData(input_cds))


umap_coords <- sc.umap[colnames(sc.data.subset), c('UMAP1','UMAP2')]

head(umap_coords)


colnames(umap_coords) <- NULL




cicero_cds <- make_cicero_cds(input_cds, reduced_coordinates = umap_coords, k=30)
window <- 1e6
data('human.hg19.genome')
distance_parameters <- estimate_distance_parameter(cicero_cds, window=window, maxit=100, sample_num=100, distance_constraint=500000, genomic_coords=human.hg19.genome)
mean_distance_parameter <- mean(unlist(distance_parameters))
cicero_out <- generate_cicero_models(cicero_cds, distance_parameter=mean_distance_parameter, window=window, genomic_coords=human.hg19.genome)
conns <- assemble_connections(cicero_out, silent=FALSE)


,site_name,chr,bp1,bp2,num_cells_expressed
,<fct>,<chr>,<dbl>,<dbl>,<int>
chr1_100003518_100003891,chr1_100003518_100003891,1,100003518,100003891,109
chr1_100009551_100010651,chr1_100009551_100010651,1,100009551,100010651,192
chr1_100012339_100012539,chr1_100012339_100012539,1,100012339,100012539,85
chr1_100014582_100014865,chr1_100014582_100014865,1,100014582,100014865,128
chr1_100019342_100019752,chr1_100019342_100019752,1,100019342,100019752,66
chr1_100024645_100025217,chr1_100024645_100025217,1,100024645,100025217,92


,UMAP1,UMAP2
,<dbl>,<dbl>
D032_AACGAGAGCTAACCACCTAAAG,5.466162,3.883028
D032_AACGAGAGCTAACCATGCATGA,-12.578452,-2.374089
D032_AACGAGAGCTAACGTGATTTGG,4.399629,4.022728
D032_AACGAGAGCTAAGCTCAACGCA,7.654104,3.517541
D032_AACGAGAGCTACAGTTAACGGT,-7.315311,8.959862
D032_AACGAGAGCTACCACGCAAACT,8.069730,2.221157


Overlap QC metrics:
Cells per bin: 30
Maximum shared cells bin-bin: 26
Mean shared cells bin-bin: 0.00954767770597563
Median shared cells bin-bin: 0
Warning message in estimate_distance_parameter(cicero_cds, window = window, maxit = 100, :
"Could not calculate sample_num distance_parameters - see documentation details"

[1] "Successful cicero models:  354"
[1] "Other models: "

  Too many elements in range Zero or one element in range 
                         104                         5885 
[1] "Models with errors:  0"


In [25]:

saveRDS(conns, file.path(wd, paste0(outConnectionFilePrefix, "pooled", outRdsConnsSuffix)))
write.table(conns, file.path(wd, paste0(outConnectionFilePrefix, "pooled", outConnsSuffix)), sep='\t', quote=FALSE, row.names=FALSE)

## this step is to remove duplicated connections
conns = conns[order(-conns$coaccess),]
bed = cbind(str_split_fixed(conns[,1], "\\_", 3 ), str_split_fixed(conns[,2], "\\_", 3 ))
ord = matrix(parRapply(clus, bed, function(x) x[order(x)] ), ncol=6, byrow=T)
ord = cbind(ord[, c(5,1:2,5,3:4)], conns$coaccess)
dedup = ord[!duplicated(ord),]
dedup = data.frame( Peak1 = paste(dedup[,1], dedup[,2], dedup[,3], sep="_")  , 
                    Peak2 = paste(dedup[,4], dedup[,5], dedup[,6], sep="_") , coaccess = dedup[,7]  )


write.table(dedup, file.path(wd, paste0(outConnectionFilePrefix, "pooled", outConnsDedupSuffix)), sep='\t', quote=FALSE, row.names=FALSE)
